In [ ]:
import sqlite3
import pandas as pd

conn=sqlite3.connect(r'C:\Users\hp\Downloads\chinook (1).db')
cursor=conn.cursor()
deta=cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
for tup in deta:
    print(f"{tup[0]} table")
    print(pd.read_sql_query(f"select * from {tup[0]}", conn))
    print("\n\n\n")




albums table
     AlbumId                                              Title  ArtistId
0          1              For Those About To Rock We Salute You         1
1          2                                  Balls to the Wall         2
2          3                                  Restless and Wild         2
3          4                                  Let There Be Rock         1
4          5                                           Big Ones         3
..       ...                                                ...       ...
342      343                             Respighi:Pines of Rome       226
343      344  Schubert: The Late String Quartets & String Qu...       272
344      345                                Monteverdi: L'Orfeo       273
345      346                              Mozart: Chamber Music       274
346      347  Koyaanisqatsi (Soundtrack from the Motion Pict...       275

[347 rows x 3 columns]




sqlite_sequence table
            name   seq
0         genres    25
1  

In [3]:
#Har bir mijoz tomonidan xaridlar uchun sarflangan jami summani toping (hisob-fakturalarni inobatga olgan holda).
al=pd.read_sql_query("select * from albums", conn)
print(al)
n=pd.read_sql_query("select * from sqlite_sequence", conn)
print(n)

     AlbumId                                              Title  ArtistId
0          1              For Those About To Rock We Salute You         1
1          2                                  Balls to the Wall         2
2          3                                  Restless and Wild         2
3          4                                  Let There Be Rock         1
4          5                                           Big Ones         3
..       ...                                                ...       ...
342      343                             Respighi:Pines of Rome       226
343      344  Schubert: The Late String Quartets & String Qu...       272
344      345                                Monteverdi: L'Orfeo       273
345      346                              Mozart: Chamber Music       274
346      347  Koyaanisqatsi (Soundtrack from the Motion Pict...       275

[347 rows x 3 columns]
            name   seq
0         genres    25
1    media_types     5
2        artists   

In [6]:
import pandas as pd
import sqlite3

# Ma'lumotlar bazasiga ulanish
conn = sqlite3.connect(r'C:\Users\hp\Downloads\chinook (1).db')  # yoki sizdagi bazaning nomi

# So‘rov: mijozlar bo‘yicha umumiy sarf
query = """
SELECT 
    c.CustomerId,
    c.FirstName || ' ' || c.LastName AS CustomerName,
    SUM(i.Total) AS TotalSpent
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId

ORDER BY TotalSpent DESC;
"""

# So‘rovni bajarish
df = pd.read_sql_query(query, conn)
print(df)


   CustomerId    CustomerName  TotalSpent
0           1  Luís Gonçalves      2328.6


In [1]:
import pandas as pd

# Customers jadvali
customers_df = pd.DataFrame({
    'CustomerId': [1, 2, 3, 4, 5, 6],
    'FirstName': ['Luís', 'Leonie', 'François', 'Bjorn', 'František', 'Helena'],
    'LastName': ['Gonçalves', 'Köhler', 'Tremblay', 'Hansen', 'Wichterlová', 'Holý']
})

# Invoices jadvali (Total = invoice summasi)
invoices_df = pd.DataFrame({
    'InvoiceId': [1, 2, 3, 4, 5, 6, 7],
    'CustomerId': [1, 2, 3, 2, 1, 3, 1],
    'Total': [10.0, 15.5, 8.0, 9.5, 12.0, 6.0, 20.0]
})


In [2]:
# JOIN: invoices bilan customers'ni birlashtiramiz
merged_df = pd.merge(invoices_df, customers_df, on='CustomerId')

# Foydalanuvchi to'liq ismini yaratamiz
merged_df['FullName'] = merged_df['FirstName'] + ' ' + merged_df['LastName']


In [ ]:
import pandas as pd

# Assume CSVs or dataframes already loaded:
# customers_df, invoices_df, invoice_items_df, tracks_df

# Join invoice_items with invoices to get CustomerId
invoice_items_merged = invoice_items_df.merge(invoices_df, on='InvoiceId')

# Join with tracks to get AlbumId
invoice_items_merged = invoice_items_merged.merge(tracks_df[['TrackId', 'AlbumId']], on='TrackId')
# Count tracks bought by each customer per album
customer_album_purchases = invoice_items_merged.groupby(['CustomerId', 'AlbumId'])['TrackId'].nunique().reset_index(name='TracksPurchased')

# Get total number of tracks in each album
tracks_per_album = tracks_df.groupby('AlbumId')['TrackId'].nunique().reset_index(name='TotalTracksInAlbum')

# Merge to compare purchases vs album total
customer_album_purchases = customer_album_purchases.merge(tracks_per_album, on='AlbumId')
# Add flag: did the customer buy the full album?
customer_album_purchases['BoughtFullAlbum'] = customer_album_purchases['TracksPurchased'] >= customer_album_purchases['TotalTracksInAlbum']
# For each customer, did they EVER buy a full album?
full_album_flags = customer_album_purchases.groupby('CustomerId')['BoughtFullAlbum'].any().reset_index()

# Customers who NEVER bought full albums = prefer individual tracks
individual_track_buyers = full_album_flags[full_album_flags['BoughtFullAlbum'] == False]

# Calculate percentage
total_customers = customers_df['CustomerId'].nunique()
individual_track_buyers_count = individual_track_buyers['CustomerId'].nunique()

percentage = (individual_track_buyers_count / total_customers) * 100

print(f"📊 Percentage of customers who prefer individual tracks: {percentage:.2f}%")


In [ ]:
import pandas as pd

# Example: load from CSVs (replace with actual file paths or data)
# customers_df = pd.read_csv('customers.csv')
# invoices_df = pd.read_csv('invoices.csv')
# invoice_items_df = pd.read_csv('invoice_items.csv')
# tracks_df = pd.read_csv('tracks.csv')

# 1️⃣ Merge invoice_items with invoices to get CustomerId per TrackId
invoice_items_with_customer = invoice_items_df.merge(invoices_df, on='InvoiceId')

# 2️⃣ Add AlbumId via TrackId from tracks_df
invoice_items_with_album = invoice_items_with_customer.merge(tracks_df[['TrackId', 'AlbumId']], on='TrackId')

# 3️⃣ Count how many tracks each customer bought per album
customer_album_tracks = (
    invoice_items_with_album
    .groupby(['CustomerId', 'AlbumId'])['TrackId']
    .nunique()
    .reset_index(name='TracksPurchased')
)

# 4️⃣ Get total number of tracks in each album
total_tracks_per_album = (
    tracks_df
    .groupby('AlbumId')['TrackId']
    .nunique()
    .reset_index(name='TotalTracksInAlbum')
)

# 5️⃣ Merge to compare purchased vs available tracks
customer_album_tracks = customer_album_tracks.merge(total_tracks_per_album, on='AlbumId')

# 6️⃣ Flag whether customer bought full album or only a subset
customer_album_tracks['BoughtFullAlbum'] = (
    customer_album_tracks['TracksPurchased'] >= customer_album_tracks['TotalTracksInAlbum']
)

# 7️⃣ Group by customer: check if they EVER bought a full album
customer_behavior = (
    customer_album_tracks
    .groupby('CustomerId')['BoughtFullAlbum']
    .any()
    .reset_index()
)

# 8️⃣ Filter customers who NEVER bought a full album
individual_track_buyers = customer_behavior[customer_behavior['BoughtFullAlbum'] == False]

# 9️⃣ Calculate percentage
total_customers = customers_df['CustomerId'].nunique()
individual_track_customer_count = individual_track_buyers['CustomerId'].nunique()

percentage = (individual_track_customer_count / total_customers) * 100

print(f"📊 Percentage of customers who prefer individual tracks: {percentage:.2f}%")
